In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Named Entity Recognition
In Natural Language Processing (NLP) an Entity Recognition is one of the common problem. The entity is referred to as the part of the text that is interested in. In NLP, NER is a method of extracting the relevant information from a large corpus and classifying those entities into predefined categories such as location, organization, name and so on. 
Information about lables: 
* geo = Geographical Entity
* org = Organization
* per = Person
* gpe = Geopolitical Entity
* tim = Time indicator
* art = Artifact
* eve = Event
* nat = Natural Phenomenon

        1. Total Words Count = 1354149 
        2. Target Data Column: Tag

In [1]:
!pip install python-crfsuite
!pip install seqeval
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-s0rlll8l
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-s0rlll8l
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=30ae8312dc0e094af98408e880313cc83fae1dfd7375c3846f09d7e8a37ba492
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ldnopfc/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [5]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 72251, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 72251 (delta 131), reused 154 (delta 53), pack-reused 71957
Receiving objects: 100% (72251/72251), 55.10 MiB | 23.99 MiB/s, done.
Resolving deltas: 100% (51248/51248), done.


In [6]:
%cd transformers
!pip install .
!pip install -r ./examples/requirements.txt
%cd ..

/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 6.4MB/s 
     |████████████████████████████████| 901kB 23.5MB/s 
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2217141 sha256=107e6b457c19f9d5981bcf69029e8325f337f77399b2d68d8f92732c081f7116
  Stored in directory: /tmp/pip-ephem-wheel-cache-17glilyh/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
ERROR: Could not open requirements file: [Errno 2] No such file or directory: './examples/requirements.txt'
/content


In [7]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2021-05-10 19:08:04--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2021-05-10 19:08:05 (36.6 MB/s) - ‘preprocess.py’ saved [991/991]



In [2]:
MAX_LENGTH = 120 #@param {type: "integer"}
MODEL = "bert-base-multilingual-cased" #@param ["bert-base-multilingual-cased"]

#### Importing Libraries

In [3]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from keras.models import Model, Input
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import pycrfsuite
from sklearn.metrics import f1_score
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from keras.preprocessing.text import text_to_word_sequence
import pickle

In [4]:
#Reading the csv file
xtrain = pd.read_csv("/content/drive/MyDrive/NLP/2Competição/train.conll", sep = ' ', skip_blank_lines=False)

In [5]:
from sklearn.model_selection import train_test_split

train, dev = train_test_split(xtrain, test_size=0.210014 , shuffle=False)

In [6]:
final = pd.DataFrame(train.O)

In [7]:
final.dtypes

O    object
dtype: object

In [27]:
final.to_csv('/content/drive/MyDrive/NLP/2Competição/teste.txt', header=False, index=False)

In [8]:
!cat /content/drive/MyDrive/NLP/2Competição/trainC.conll | cut -d " " -f 1,3 > train_temp.txt
!cat /content/drive/MyDrive/NLP/2Competição/testC.conll | cut -d " " -f 1,3 > dev_temp.txt

In [9]:
!python3 preprocess.py train_temp.txt $MODEL $MAX_LENGTH > train.txt
!python3 preprocess.py dev_temp.txt $MODEL $MAX_LENGTH > dev.txt

2021-05-10 19:30:01.867593: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-10 19:30:19.853655: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [10]:
MAX_LENGTH = 128 #@param {type: "integer"}
MODEL = "bert-base-multilingual-cased" #@param ["bert-base-multilingual-cased"]
OUTPUT_DIR = "bert-base-ml-ner" #@param ["bert-base-ml-ner"]
BATCH_SIZE = 32 #@param {type: "integer"}
NUM_EPOCHS = 3 #@param {type:"integer"}
SAVE_STEPS = 100 #@param {type: "integer"}
LOGGING_STEPS = 100 #@param {type: "integer"}
SEED = 42 #@param {type: "integer"}

In [36]:
!pip install transformers==2.0.0

     |████████████████████████████████| 296kB 6.5MB/s 
     |████████████████████████████████| 133kB 8.9MB/s 
     |████████████████████████████████| 1.2MB 8.3MB/s 
     |████████████████████████████████| 7.5MB 18.5MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
ERROR: botocore 1.20.69 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: transformers 4.6.0.dev0
    Uninstalling transformers-4.6.0.dev0:
      Successfully uninstalled transformers-4.6.0.dev0


In [45]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-8lzsa_dd
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-8lzsa_dd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2217141 sha256=2bd86a70072714095be92150aad68786e4ec58af671baa9771a379a1eb580103
  Stored in directory: /tmp/pip-ephem-wheel-cache-eoprtbm0/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Found existing installation: transformers 2.0.0
    Uninstalling transformers-2.0.0:
      Successfully uninstalled transformers-2.0.0


In [11]:
python3 run_ner.py \
  --data_dir ./ \
  --model_type bert \
  --labels .//content/drive/MyDrive/NLP/2Competição/teste.txt \
  --model_name_or_path $MODEL \
  --output_dir $OUTPUT_DIR \
  --max_seq_length  $MAX_LENGTH \
  --num_train_epochs $NUM_EPOCHS \
  --per_gpu_train_batch_size $BATCH_SIZE \
  --save_steps $SAVE_STEPS \
  --logging_steps $LOGGING_STEPS \
  --seed $SEED \
  --do_train \
  --do_eval \
  --do_predict \
  --overwrite_output_dir

SyntaxError: ignored

In [42]:
-#Display first 10 rows
train

,E,O
0,M,O
1,E,O
2,N,O
3,T,O
4,A,O
...,...,...
248354,2,O
248355,desta,O
248356,instrução,O
248357,.,O


In [15]:
df.describe()

,E,O
count,306759,306833
unique,20941,13
top,",",O
freq,23079,265104
